In [1]:
from __future__ import division

import numpy as np
import xgboost as xgb
import pickle
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

from tqdm import tqdm
import printTime as pt
import gc

import warnings
import random
warnings.filterwarnings('ignore')
pt.printTime()

2019-08-21 14:49:33


In [2]:
age_test = pd.read_csv("../data/age_test.csv", header = None)
age_train = pd.read_csv("../data/age_train.csv", header = None)
user_app_usage = pd.read_csv("../data/user_app_usage.csv", nrows=500000, header = None)

In [3]:
# date_dict = {'2019-02-28': 0}
# for date in user_app_usage.loc[:, 4]:
#     if date not in date_dict:
#         date_dict[date] = int(date.split('-')[2])
# len(date_dict)

date_dict = {}
i = 0
for one_day in tqdm(user_app_usage.loc[:, 4]):
    if one_day not in date_dict.keys():
        date_dict[one_day] = i
        i += 1
day_num = i
pt.printTime()
len(date_dict)

100%|██████████| 500000/500000 [00:00<00:00, 2306920.26it/s]

2019-08-21 14:49:34


30

In [4]:
user_app_usage = pd.read_csv("../data/user_app_usage.csv", header = None)
user_app_usage = user_app_usage.values

# top 100 app_keys 30 days usage time

In [5]:
app_keys = np.loadtxt('../processed/high_attractive_app_keys.txt', dtype=str, delimiter=',')
app_keys = app_keys[:100]
app_keys.shape
app_keys = app_keys.tolist()

In [6]:
all_users = pd.merge(age_train.drop([1], axis = 1), age_test, on=0, how='outer')
all_users.shape

(2512500, 1)

In [7]:
add_columns = []
for i in range(len(app_keys)):
    for j in range(30):
        add_columns.append(i*30+j)
len(add_columns)

3000

In [8]:
add_columns = np.zeros((all_users.shape[0], len(add_columns)))

In [9]:
uid_index_dict = {}
index = 0
for uid in all_users[0]:
    uid_index_dict[int(uid)] = index 
    index += 1

In [10]:
all_users = all_users.values

In [11]:
for i in tqdm(range(user_app_usage.shape[0])):
    if user_app_usage[i, 1] in app_keys:
        if user_app_usage[i, 2] > 18426:
            add_columns[uid_index_dict[user_app_usage[i, 0]], app_keys.index(user_app_usage[i, 1])*30+date_dict[user_app_usage[i, 4]]] = 18426
        else:
            add_columns[uid_index_dict[user_app_usage[i, 0]], app_keys.index(user_app_usage[i, 1])*30+date_dict[user_app_usage[i, 4]]] = user_app_usage[i, 2]

100%|██████████| 651007719/651007719 [37:32<00:00, 289018.33it/s]  


In [12]:
del user_app_usage
gc.collect()
pt.printTime()

2019-08-21 15:51:41


# 转换为csr稀疏矩阵,并保存

In [13]:
from sklearn import preprocessing
from scipy.sparse import csr_matrix, hstack
from scipy import sparse

In [14]:
add_columns = csr_matrix(add_columns)

In [15]:
scale = preprocessing.MaxAbsScaler()
add_columns = scale.fit_transform(add_columns)
pt.printTime()

2019-08-21 15:57:04


In [16]:
trainData = add_columns[:2010000]
testData = add_columns[2010000:]
pt.printTime()

2019-08-21 15:57:11


In [17]:
sparse.save_npz('../trainTestData/trainData3000.npz', trainData)  #保存
sparse.save_npz('../trainTestData/testData3000.npz', testData)  #保存

In [18]:
pt.printTime()

2019-08-21 16:00:38
